In [ ]:
!python -m spacy download en_core_web_md
!pip install openpyxl
!pip install transformers
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

In [3]:
import pandas as pd
import spacy
from spacy.tokens.token import Token
import string

from typing import List, Set
nlp = spacy.load("en_core_web_md")

In [ ]:
def convert_to_string(data) -> str:
    return data if isinstance(data, str) else str(data)

def tokenize(text: str) -> List[Token]:
  doc = nlp(text)
  return [w for sent in doc.sents for w in sent]

def remove_punctuation(tokens: List[Token]) -> List[Token]:
  return [t for t in tokens if t.text not in string.punctuation]

def remove_stop_words(tokens: List[Token]) -> List[Token]:
  return [t for t in tokens if not t.is_stop]

def lemmatize(tokens: List[Token]) -> List[str]:
  return [t.lemma_ for t in tokens]

def case_fold(tokens: List[str]) -> List[str]:
    return [t.lower() for t in tokens]

def pre_process_text(text: str) -> List[str]:
    return case_fold(lemmatize(remove_punctuation(tokenize(convert_to_string(text)))))

def get_num_sentences(data) -> int:
    doc = nlp(data if isinstance(data, str) else str(data))
    return len(list(doc.sents))

def get_num_words(data) -> int:
    doc = nlp(data if isinstance(data, str) else str(data))
    sentence_length = 0
    for sent in doc.sents: sentence_length += len(sent)
    return sentence_length


In [7]:
# conservative_df = pd.read_csv('cons_comments.csv', header=0)
# liberal_df = pd.concat([pd.read_excel('lib_comments.xlsx', header=0), pd.read_excel('dem_comments.xlsx', header=0)])
conservative_df = pd.read_csv('conservative_metadata_and_preprocessed_with_stop_words.csv', header=0)
liberal_df = pd.read_csv('liberal_metadata_preprocessed_with_stop_words.csv', header=0)

In [ ]:
conservative_df['preprocess_body'] = conservative_df['body'].apply(pre_process_text)
conservative_df['num_sentences'] = conservative_df['body'].apply(get_num_sentences)
conservative_df['numWords'] = conservative_df['body'].apply(get_num_words)
conservative_df.to_csv('conservative_metadata_and_preprocessed_with_stop_words.csv', index = False)

liberal_df['preprocess_body'] = liberal_df['body'].apply(pre_process_text)
liberal_df['num_sentences'] = liberal_df['body'].apply(get_num_sentences)
liberal_df['numWords'] = liberal_df['body'].apply(get_num_words)
liberal_df.to_csv('liberal_metadata_preprocessed_with_stop_words.csv', index = False)
# random_sample_size = 194944
# conservative_base_sample_df = conservative_df.sample(n=194944)
# conservative_base_sample_df.to_csv('conservative_base_sample.csv', index = False)
# liberal_base_sample_df = liberal_df
# conservative_2k_sample = conservative_base_sample_df.sample(n=2000)
# liberal_2k_sample = liberal_base_sample_df.sample(n=2000)
# conservative_2k_sample.to_csv('conservative_2k_sample.csv', index = False)
# liberal_2k_sample.to_csv('liberal_2k_sample.csv', index = False)

In [ ]:
def create_vocab(data: List[List[str]]) -> Set[str]:
    data = list(map(lambda x: x.strip('[]').split(",") if type(x) == str else [], data))
    vocab = {str(token).lower() for tokens in data for token in tokens}
    return vocab

def find_common_vocab(vocab1, vocab2):
    return list(vocab1.intersection(vocab2))

In [ ]:
conservative_vocab = create_vocab(conservative_df['preprocess_body'])
liberal_vocab = create_vocab(liberal_df['preprocess_body'])

common_vocab = find_common_vocab(conservative_vocab, liberal_vocab)
conservative_vocab_percent = len(conservative_vocab) / len(common_vocab) * 100
liberal_vocab_percent = len(liberal_vocab) / len(common_vocab) * 100

In [6]:
import collections

def get_meta_data(dataframe):
    word_to_doc_map = collections.defaultdict(set)
    word_to_score_map = collections.defaultdict(int)
    word_to_freq_map = collections.defaultdict(int)
    
    for index, row in dataframe.iterrows():
        doc_id, score = row["id"], row["score"]
        try:
            for word in row["preprocess_body"].strip('[]').split(','):
                word_to_doc_map[word].add(doc_id)
                word_to_score_map[word] += score
                word_to_freq_map[word] += 1
        except Exception as e:
            print("Row: ", row["preprocess_body"])
            print("The row might be a non string value: ", e)
    
    meta_data = {"word": [], "freq": [], "score": [], "doc_ids": []}
    for word, freq in word_to_freq_map.items():
        meta_data["word"].append(word)
        meta_data["freq"].append(freq)
        meta_data["score"].append(word_to_score_map.get(word, 0))
        meta_data["doc_ids"].append(word_to_doc_map.get(word, []))
        
    return pd.DataFrame(meta_data)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

x = ['conservative', 'liberal']
colors = ['red', 'blue']
size = [len(conservative_df['body']), len(liberal_df['body'])]
vocab_percent = [conservative_vocab_percent , liberal_vocab_percent]
vocab_size = [len(conservative_vocab), len(liberal_vocab)]
average_sent_length = [np.mean(conservative_df['num_sentences']),np.mean(liberal_df['num_sentences'])]
average_word_count = [np.mean(conservative_df['numWords']), np.mean(liberal_df['numWords'])]

# post size
plt.bar(x, size, color=colors)
plt.xlabel('Category')
plt.ylabel('Number of Posts')
plt.title('Reddit Posts size comparison')
for i, v in enumerate(size):
    plt.text(i, v + 1, str(v), color='black', fontweight='bold', ha='center')
plt.show()

# vocabulary distribution
chart = plt.pie(vocab_percent, labels=x, autopct='%1.1f%%', colors=colors)
plt.title('Vocabulary Distribution')
legend_labels = []
for i in range(len(vocab_size)):
    legend_labels.append(f"{x[i]}: {vocab_size[i]}")
plt.legend(chart[0], legend_labels, title='Category', loc='center left')
plt.show()

# Sentence length
plt.barh(x, average_sent_length, color=colors)
plt.xlabel('Number of sentences per post')
plt.ylabel('Category')
plt.title('Average Sentences per Post')
for i, v in enumerate(average_sent_length):
    plt.text(v + 0.3, i, str(round(v, 2)), color='black', fontweight='bold', ha='center')
plt.show()

# Number of words
plt.scatter(x, average_word_count, s=[a * 25 for a in average_word_count], alpha=0.5, c=colors, cmap='viridis')
plt.xlabel('Category')
plt.ylabel('Number of words per post')
plt.title('Average word count')
for i, v in enumerate(average_word_count):
    plt.text(i, v, str(int(v)), color='black', fontweight='bold', ha='center')
plt.show()


In [ ]:
conservative_words = get_meta_data(conservative_df).sort_values('score', ascending = False)["word"].head(100)
liberal_words = get_meta_data(liberal_df).sort_values('score', ascending = False)["word"].head(100)
intersection = set(conservative_words).intersection(set(liberal_words))
print(intersection)

In [10]:
words_of_interest = ['conservative']
# ,'trump','democrat','election','party','job','news','vote','president','american','right','pay','run','government','country','democrats','bernie','biden','state','life','support']

In [11]:
from collections import defaultdict

similar_sentences_group = {}

conservative_metadata = get_meta_data(conservative_df)
liberal_metadata = get_meta_data(liberal_df)
def padWord(word): return "'" + str(word) + "'"


for word in list(map(lambda x: padWord(x), words_of_interest)):
    similar_sentences_group[word] = defaultdict(list)
    conservative_docs = list(set([postId 
                                  for postSet in conservative_metadata[conservative_metadata["word"] == word]["doc_ids"]
                                  for postId in postSet]
                                )
                            )
    conservative_docs = list(map(lambda x: x.strip("'"), conservative_docs))
    for index, row in conservative_df[conservative_df["id"].isin(conservative_docs)].iterrows():
        doc = nlp(row["body"])
        similar_sentences_group[word]["conservative"].extend([sent for sent in doc.sents])
     
    
    liberal_docs = list(set([postId 
                             for postSet in liberal_metadata[liberal_metadata["word"] == word]["doc_ids"]
                             for postId in postSet]
                           )
                       )
    liberal_docs = list(map(lambda x: x.strip("'"), liberal_docs))
    for index, row in liberal_df[liberal_df["id"].isin(liberal_docs)].iterrows():
        doc = nlp(row["body"])
        similar_sentences_group[word]["liberal"].extend([sent for sent in doc.sents])

Row:  nan
The row might be a non string value:  'float' object has no attribute 'strip'
Row:  nan
The row might be a non string value:  'float' object has no attribute 'strip'


In [59]:
import torch
import numpy as np
from transformers import BertTokenizer, BertModel
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased', output_hidden_states = True)
model.eval()

def preprocess_cosine_similarity(sent) -> np.array:
    marked_text = "[CLS] " + sent.text + " [SEP]"
    tokenized_text = tokenizer.tokenize(marked_text)
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
    segments_ids = [1] * len(tokenized_text)
    tokens_tensor = torch.tensor([indexed_tokens])
    segments_tensor = torch.tensor([segments_ids])
    with torch.no_grad():
        outputs = model(tokens_tensor, segments_tensor)
        hidden_states = outputs[2]
        token_vecs = hidden_states[-2][0]
        sentence_embedding = torch.mean(token_vecs, dim=0)
        return sentence_embedding

import torch.nn.functional as F

def cosine_similarity(vector_1, vector_2):
    return F.cosine_similarity(vector_1, vector_2, dim=0).item()

def find_most_similar_document_cosine(sentence_df, data_point) -> float:
  sentence_df["similarity_score"] = sentence_df["sentence_processed"].apply(lambda x: cosine_similarity(x, data_point))
  return sentence_df.iloc[sentence_df["similarity_score"].idxmax()].sentence

bert-base-uncased
https://huggingface.co/bert-base-uncased/resolve/main/vocab.txt
etag tracker:  "fb140275c155a9c7c5a3b3e0e77a9e839594a938"
45c3f7a79a80e1cf0a489e5c62b43f173c15db47864303a55d623bb3c96f72a5
https://huggingface.co/bert-base-uncased/resolve/main/added_tokens.json
https://huggingface.co/bert-base-uncased/resolve/main/special_tokens_map.json
https://huggingface.co/bert-base-uncased/resolve/main/tokenizer_config.json
etag tracker:  "a661b1a138dac6dc5590367402d100765010ffd6"
c1d7f0a763fb63861cc08553866f1fc3e5a6f4f07621be277452d26d71303b7e
https://huggingface.co/bert-base-uncased/resolve/main/tokenizer.json
etag tracker:  "949a6f013d67eb8a5b4b5b46026217b888021b88"
534479488c54aeaf9c3406f647aa2ec13648c06771ffe269edabebd4c412da1d
https://huggingface.co/bert-base-uncased/resolve/main/config.json
etag tracker:  "45a2321a7ecfdaaf60a6c1fd7f5463994cc8907d"
3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e
https://huggingface.co/bert-base-uncased/resolve/main/config.json

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
import warnings
warnings.filterwarnings("ignore")

for word in similar_sentences_group:
    conservative_bucket_df = pd.DataFrame({"sentence": similar_sentences_group[word]["conservative"]})
    conservative_bucket_df["sentence_processed"] = conservative_bucket_df["sentence"].apply(preprocess_cosine_similarity)
    conservative_final_df = pd.DataFrame(columns=["sentence", "similar_sentence"])
    for index, row in conservative_bucket_df.iterrows():
        conservative_final_df = pd.concat([conservative_final_df,
                              pd.DataFrame(
                                  [(row["sentence"], 
                                   find_most_similar_document_cosine(conservative_bucket_df[conservative_bucket_df.index != index].reset_index(), row["sentence_processed"]))
                                  ], columns=["sentence", "similar_sentence"]
                                )
                              ]
                            )
    
    
    liberal_bucket_df = pd.DataFrame({"sentence": similar_sentences_group[word]["liberal"]})
    liberal_bucket_df["sentence_processed"] = liberal_bucket_df["sentence"].apply(preprocess_cosine_similarity)
    liberal_final_df = pd.DataFrame(columns=["sentence", "similar_sentence"])
    for index, row in liberal_bucket_df.iterrows():
        liberal_final_df = pd.concat([liberal_final_df,
                              pd.DataFrame(
                                  [(row["sentence"], 
                                   find_most_similar_document_cosine(liberal_bucket_df[liberal_bucket_df.index != index].reset_index(), row["sentence_processed"]))
                                  ], columns=["sentence", "similar_sentence"]
                                )
                              ]
                            )
    
conservative_final_df.to_csv("similar_sents_conservative.csv",index = False)
liberal_final_df.to_csv("similar_sents_liberal.csv", index = False)
    

In [ ]:


# device_name = tf.test.gpu_device_name()
# if torch.cuda.is_available():    
#     device = torch.device("cuda")
#     print('There are %d GPU(s) available.' % torch.cuda.device_count())
#     print('We will use the GPU:', torch.cuda.get_device_name(0))
# else:
#     print('No GPU available, using the CPU instead.')
#     device = torch.device("cpu")
    
